## SCRAP (export en df)

In [15]:


import json

# pour exploiter les requêtes
from requests import post

# pour le contrôle des requêtes
from time import sleep, time
from random import randint
from abc import ABCMeta, abstractmethod
from warnings import warn



def convert_arr_2_string(arr, sep):
    """ Convert array to string with separator """
    return sep.join(arr)

def get_term(path):
    """ get term in a path. Otherwise, return 'Non renseigné' """
    if path is not None:
        return path.text.strip()
    return ''

def jprint(obj):
    """ convert array to json """
    # create a formatted string of the Python JSON object
    return json.dumps(obj, sort_keys=True) #, indent=4 

def post_data(root_path, payload, requests, start_time):
    """ post data and get the result  """
    response = post(root_path, json=payload)
    content = response.content
    
    ### pause de 8 à 15s
    sleep(randint(8, 15))
    
    ### afficher les informations sur les requêtes
    requests += 1 # incrémentation du nombre de requête
    elapsed_time = time() - start_time
    
    ### avertir si le code status est différent de 200
    if response.status_code != 200:
        warn('Request: {}; Status code:{}'.format(requests, requests/elapsed_time))
    
    ### stopper quand les requêtes atteignent le quota
    if requests > 200:
        warn('Nombre de requêtes trop important')
        return
    
    try:
        json_data = json.loads(content)
    except:
        json_data = ""
    
    return json_data


class scraping_jobs(metaclass=ABCMeta):
    
    def __init__(self, s_job, type_contract):
        self.s_job = s_job
        self.type_contract = type_contract
    
    @abstractmethod
    def scrap_job(self, dict_jobs, s_job, type_contract):
        pass


class scraping_jobs_apec(scraping_jobs):
    
    #
    def set_code_dpt(self, code_dpt):
        self.code_dpt = code_dpt
    
    #
    def scrap_job(self):
        ### paramètres pris
        param_search_words = self.s_job 
        
        ### pages à parcourir
        pages = [str(i) for i in range(0, 3)]  #### le nombre de pages voulu 
        requests = 0
        start_time = time()
        
        dict_jobs = []
        
        ### parcours des pages
        for page in pages:
            #
            root_path = 'https://www.apec.fr/cms/webservices/rechercheOffre'
            payload = {
                'typeClient': 'CADRE',
                'sorts' : [{
                    'type': 'SCORE',
                    'direction': 'DESCENDING'
                }],
                'pagination': {
                    'range': 20,
                    'startIndex': page
                },
                'activeFiltre': True,
                'pointGeolocDeReference': {
                    'distance': 0
                },
                'motsCles': param_search_words
            }
            
            json_data = post_data(root_path, payload, requests, start_time)
            
            ### vérifier l'existence de l'index 'resultats'
            if 'resultats' in json_data:
                result_containers = json_data['resultats']
            
                ### extraction des données du JSON renvoyé
                for result in result_containers:
                    #
                    if result['localisable'] == True:
                        latitude = result['latitude'],
                        longitude = result['longitude'],
                    else:
                        latitude = 0
                        longitude = 0
                    dict_jobs.append({
                        'intitule' : result['intitule'],
                        'link' : 'https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/'+result['numeroOffre'],
                        'numero_offre' : result['numeroOffre'],
                        'lieuTexte' : result['lieuTexte'],
                        'texteOffre' : result['texteOffre'],
                        'nomCommercial' : result['nomCommercial'],
                        'score' : result['score'],
                        'salaireTexte' : result['salaireTexte'],
                        'typeContrat' : result['typeContrat'],
                        'datePublication' : result['datePublication'],
                        'dateValidation' : result['dateValidation'],
                        'latitude' : latitude,
                        'longitude' : longitude,
                        'contractDuration' : result['contractDuration'],
                    })
            
        ### retourne array
        return dict_jobs
    
    


s_job = "Consultant en recrutement- Lyon - Intérim F/H"
city = ""
code_dpt = ""
type_contract = ''

arr_jobs = []

print('please wait, search in progress...')

## apec
sjapec = scraping_jobs_apec(s_job, type_contract)
sjapec.set_code_dpt(code_dpt)
dict_tmp = sjapec.scrap_job()
if len(dict_tmp) > 0:
    arr_jobs += dict_tmp


### impression des jobs en json
print(jprint(arr_jobs))


print(len( arr_jobs))
print(type(arr_jobs))

import pandas as pd

jobs = pd.DataFrame(arr_jobs)
print(jobs)

please wait, search in progress...
[{"contractDuration": 0, "datePublication": "2024-01-03T23:05:23.000+0000", "dateValidation": "2024-01-03T23:05:23.000+0000", "intitule": "Data scientist/Data Analyst F/H", "latitude": ["48.7838513"], "lieuTexte": "V\u00e9lizy-Villacoublay - 78", "link": "https://www.apec.fr/candidat/recherche-emploi.html/emploi/detail-offre/172742239W", "longitude": ["2.1973528"], "nomCommercial": "INNOVAI TEK", "numero_offre": "172742239W", "salaireTexte": "45 - 55 k\u20ac brut annuel", "score": 46.442825, "texteOffre": "Nous cherchons :  Bac +4 en \u00e9cole d'ing\u00e9nieur ou un master 2 avec une sp\u00e9cialisation en informatique / data Science / IA / statistiques .  Exp\u00e9rience de minimum 3 ans sur les projets de Data science/Data Analyst.  Ma\u00eetrise de plusieurs langage parmi : Python, Spark, SQL .  Tres b...", "typeContrat": 101888}, {"contractDuration": 0, "datePublication": "2023-12-14T15:48:38.000+0000", "dateValidation": "2023-12-14T15:48:38.000+

In [16]:
df_apec=jobs

In [17]:
df_apec.head()

,intitule,link,numero_offre,lieuTexte,texteOffre,nomCommercial,score,salaireTexte,typeContrat,datePublication,dateValidation,latitude,longitude,contractDuration
0,Data scientist/Data Analyst F/H,https://www.apec.fr/candidat/recherche-emploi....,172742239W,Vélizy-Villacoublay - 78,Nous cherchons : Bac +4 en école d'ingénieur ...,INNOVAI TEK,46.442825,45 - 55 k€ brut annuel,101888,2024-01-03T23:05:23.000+0000,2024-01-03T23:05:23.000+0000,"(48.7838513,)","(2.1973528,)",0
1,Data Analyst / Data engineer F/H,https://www.apec.fr/candidat/recherche-emploi....,172631521W,Paris 09 - 75,Auprès d'un de nos clients dans l'univers de l...,VR CONSEIL,44.463203,45 - 75 k€ brut annuel,101888,2023-12-14T15:48:38.000+0000,2023-12-14T15:48:38.000+0000,"(48.8771639,)","(2.3374635,)",0
2,Data Analyst / Data scientist F/H,https://www.apec.fr/candidat/recherche-emploi....,172777112W,Lyon 09 - 69,La ligne de service Consulting & Solutions d'A...,AKKA SERVICES,43.519090,A négocier,101888,2024-01-08T15:35:56.000+0000,2024-01-08T15:35:56.000+0000,"(45.7826243,)","(4.8087685,)",0
3,Business Data Analyst - Data Wranglers F/H,https://www.apec.fr/candidat/recherche-emploi....,172685127W,Puteaux - 92,Le profil de Data Analyst en Risque de Crédit ...,Meteojob,41.526207,A négocier,101888,2024-01-10T00:24:53.000+0000,2024-01-10T00:24:53.000+0000,"(48.8836138,)","(2.2387286,)",0
4,Data Engineer F/H,https://www.apec.fr/candidat/recherche-emploi....,172601750W,Paris 08 - 75,Vous intégrez l'équipe Datahub de l'entreprise...,CARAZ,40.826410,60 - 70 k€ brut annuel,101888,2023-12-11T11:46:58.000+0000,2023-12-11T11:46:58.000+0000,"(48.8727215,)","(2.312556,)",0


## Creation des df sous format BDD

In [26]:
# Création du DataFrame df_lieu_apec
df_lieu_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'libelle': df_apec['lieuTexte'],
    'latitude': df_apec['latitude'],
    'longitude': df_apec['longitude'],
    'codePostal': df_apec['lieuTexte'].str[-2:],
    'commune': ''
})
df_lieu_apec['latitude'] = df_lieu_apec['latitude'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
df_lieu_apec['longitude'] = df_lieu_apec['longitude'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

df_lieu_apec.head(1)

,id,libelle,latitude,longitude,codePostal,commune
0,1,Vélizy-Villacoublay - 78,48.7838513,2.1973528,78,


In [19]:
# Création du DataFrame df_entreprise_apec
df_entreprise_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'nom': df_apec['nomCommercial'],
    'adaptee': ''
})
df_entreprise_apec.head(1)

,id,nom,adaptee
0,1,INNOVAI TEK,


In [27]:
# Création du DataFrame df_contrat_apec
df_contrat_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'typeContrat': df_apec['typeContrat'],
    'typeContratLibelle': '',
    'natureContrat': '',
    'dureeTravailLibelle': '',
    'dureeTravailLibelleConverti': '',
    'alternance': ''
})
df_contrat_apec.head(1)

,id,typeContrat,typeContratLibelle,natureContrat,dureeTravailLibelle,dureeTravailLibelleConverti,alternance
0,1,101888,,,,,


In [22]:
df_contrat_apec['typeContrat'].value_counts()


typeContrat
101888    57
101889     3
Name: count, dtype: int64

In [23]:
# Création du DataFrame df_origine_apec
df_origine_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'origine': 'APEC',
    'adaptee': ''
})
df_origine_apec.head(1)

,id,origine,adaptee
0,1,APEC,


In [24]:
# Création du DataFrame df_qualification_apec
df_qualification_apec = pd.DataFrame({
    'id': range(1, len(df_apec) + 1),
    'qualification': ''
})
df_qualification_apec.head(1)

,id,qualification
0,1,
